# Q1

## 1. 데이터 살펴보기 &  zero-padding

In [1]:
import numpy as np

In [2]:
# 이미지 데이터
image = np.array([[2,0,2,1,3,2],
                [0,2,0,2,2,2],
                [1,0,1,3,1,1],
                [0,0,1,1,1,0],
                [0,1,3,4,1,0],
                [0,1,0,0,5,2]])

#zero padding
data_pad = np.pad(image, ((1,1),(1,1)), 'constant', constant_values=0)


# filter
flt = np.array ([[1,0,0],
                 [0,1,0],
                 [0,0,1]])
#stride
stride_size = 1


print(data_pad)
image.shape, data_pad.shape, flt.shape

[[0 0 0 0 0 0 0 0]
 [0 2 0 2 1 3 2 0]
 [0 0 2 0 2 2 2 0]
 [0 1 0 1 3 1 1 0]
 [0 0 0 1 1 1 0 0]
 [0 0 1 3 4 1 0 0]
 [0 0 1 0 0 5 2 0]
 [0 0 0 0 0 0 0 0]]


((6, 6), (8, 8), (3, 3))

## 2. convolution 연산

In [3]:
#convolution 연산
def conv(image, filter, stride_size,mode= 'sum'):
    # image, filter, stride 사이즈 가져오기
    (n_H_prev, n_W_prev) = image.shape
    f = filter.shape[0]
    stride = stride_size
    
    # ouput dimension (위 데이터의 경우 (6,6))
    n_H = int(1+(n_H_prev-f)/stride)
    n_W = int(1+(n_W_prev-f)/stride)
    Z = np.zeros([n_H, n_W])
    
    for h in range(n_H):
         for w in range(n_W):
            vert_start = h*stride
            vert_end = vert_start + f
            horiz_start = w*stride
            horiz_end = horiz_start + f
   
            image_slice = image[vert_start:vert_end, horiz_start:horiz_end]
            s = image_slice*flt
            
            #pooling 모드 설정
            if mode == 'max':
                Z[h,w] = np.max(s)
            elif mode == 'sum':
                Z[h,w] = np.sum(s)

    return Z

### i) stride=(1,1)일 때

In [4]:
conv(data_pad,flt,1).astype(np.int64)

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

### ii) stride=(2,2) 적용, max pooling

In [5]:
conv(data_pad,flt,1,mode='max').astype(np.int64)

array([[2, 0, 2, 2, 3, 2],
       [0, 2, 3, 2, 2, 3],
       [1, 1, 2, 3, 2, 2],
       [1, 3, 4, 1, 3, 1],
       [1, 1, 3, 5, 2, 1],
       [0, 1, 1, 3, 5, 2]])

# Q2

## 1. 기존 모델

In [6]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cude':
    torch.cuda.manual_seed_all(777)
    

# parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100

# dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

# CNN Model
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
    

# instantiate Model
model = CNN().to(device)


# loss function
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    for x,y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))
        
print('Learning Finished') 

Learning started. It takes sometime.
[Epoch:    1] cost = 0.174603894
[Epoch:    2] cost = 0.0733581632
[Epoch:    3] cost = 0.0611300059
[Epoch:    4] cost = 0.0559642725
[Epoch:    5] cost = 0.0501051769
[Epoch:    6] cost = 0.0508755893
[Epoch:    7] cost = 0.0459941924
[Epoch:    8] cost = 0.0456465296
[Epoch:    9] cost = 0.0417811945
[Epoch:   10] cost = 0.0413998663
[Epoch:   11] cost = 0.0419416837
[Epoch:   12] cost = 0.0409877747
[Epoch:   13] cost = 0.0374196023
[Epoch:   14] cost = 0.037519075
[Epoch:   15] cost = 0.0371012986
Learning Finished


In [7]:
# Accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    model1_accuracy=accuracy.item()

Accuracy:  0.8459999561309814


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## 3. 숙제 model 
layer1의 kernel size=4, layer3의 stride=3으로 변경

### - output size 구하기

##$Output size = input size - filter size + (2*padding) \over Stride$ $+1$


In [8]:
inputs=torch.Tensor(1,1,28,28)
print(inputs.shape)

#Layer1 
conv1=torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1) #(27-2+(2*0))/2+1
pool=torch.nn.MaxPool2d(kernel_size=2, stride=2) 

out1=conv1(inputs)
out1=pool(out1)
print(out1.shape)

#Layer2
conv2=torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
out2=conv2(out1)
out2=pool(out2)
print(out2.shape)

#Layer3
conv3=torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
pool=torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
out3=conv3(out2)
out3=pool(out3)
print(out3.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 32, 13, 13])
torch.Size([1, 64, 6, 6])
torch.Size([1, 128, 2, 2])


## - 모델 수정

In [9]:
# CNN Model
class CNN2(torch.nn.Module):
    
    def __init__(self):
        super(CNN2,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
                
        self.fc1 = torch.nn.Linear(2*2*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

        self.layer4 = torch.nn.Sequential(self.fc1,
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [10]:
# instantiate Model
model = CNN2().to(device)


# loss function
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    for x,y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))
        
print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.272040784
[Epoch:    2] cost = 0.122874841
[Epoch:    3] cost = 0.125163436
[Epoch:    4] cost = 0.118887879
[Epoch:    5] cost = 0.131001875
[Epoch:    6] cost = 0.123095237
[Epoch:    7] cost = 0.123946622
[Epoch:    8] cost = 0.126143485
[Epoch:    9] cost = 0.134358525
[Epoch:   10] cost = 0.126461044
[Epoch:   11] cost = 0.132519379
[Epoch:   12] cost = 0.129242495
[Epoch:   13] cost = 0.123147905
[Epoch:   14] cost = 0.123631194
[Epoch:   15] cost = 0.144520938
Learning Finished


In [11]:
# Accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    model2_accuracy=accuracy.item()

Accuracy:  0.7993999719619751


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [12]:
model1_accuracy, model2_accuracy 

(0.8459999561309814, 0.7993999719619751)